<a href="https://colab.research.google.com/github/mhrgroup/course_python_in_3_hours/blob/main/Section%2011%3A%20TensorFlow%20Keras%20for%20Classification%20Neural%20Networks/Python11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 11: TensorFlow Keras for Classification Neural Networks
By the end of this lecture, you will be able to:
1. Develop a shallow neural network using TensorFlow Keras in Python for a classification problem
2. Develop a deep neural network using TensorFlow Keras in Python for a classification problem


# 11.1. Multi-Layer Shallow and Deep Neural Networks for Classification


---


> <img src=	"	https://i.ibb.co/FX66Gbw/11-1.png	"	width="500"/>

In [1]:
#@title Import
import numpy  as np
import pandas as pd

import sklearn.preprocessing as pg # prp or whatever you like, does not matter!
import keras.utils           as ku

In [19]:
#@title Import, loading data, split data, and calibrate data all in one!
# (copy-paste from Lecture08)
data_tr   = pd.read_csv('/content/sample_data/mnist_train_small.csv')
data_te   = pd.read_csv('/content/sample_data/mnist_test.csv')

# Pandas MinMaxScaler now needs the column names to be the same for any DataFrames
data_tr.columns = list(range(data_tr.shape[1]))
data_te.columns = list(range(data_tr.shape[1]))

datain_tr = data_tr.iloc[:,1:]
dataou_tr = data_tr.iloc[:,0:1]

datain_te = data_te.iloc[:,1:]
dataou_te = data_te.iloc[:,0:1]

fun_calibration_in = pg.MinMaxScaler(feature_range=(0,1))
fun_calibration_in.fit(datain_tr)

datain_tr_calibrated = fun_calibration_in.transform(datain_tr)
datain_te_calibrated = fun_calibration_in.transform(datain_te)

dataou_tr_categorical = ku.to_categorical(dataou_tr)
dataou_te_categorical = ku.to_categorical(dataou_te)


In [21]:
data_te

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9995,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#@title Some data information
print("Total number of training datapoints {}".format(datain_tr.values.shape[0]))
print("Total number of training datapoints {}".format(datain_te.values.shape[0]))

AttributeError: ignored

In [ ]:
#@title Import TensorFlow
import tensorflow as tf

In [ ]:
#@title Define neural network parameters
in_neurons = datain_tr_calibrated.shape[1]
ou_neurons = dataou_tr_categorical.shape[1]

hn_neurons = [100,50,25]

ac_fun     = ['relu', 'relu', 'relu', 'softmax']

ls_fun     = 'categorical_crossentropy'
op_val     = 'adam'
it_val     = 100
bt_size    = 128

sh_val     = True # shuffle
vr_val     = 1    # learning to be printed

In [ ]:
#@title Construct the neural network

net        = tf.keras.models.Sequential()
net.add(tf.keras.layers.Dense(units=hn_neurons[0], activation=ac_fun[0], input_dim = in_neurons))
net.add(tf.keras.layers.Dense(units=hn_neurons[1], activation=ac_fun[1]))
net.add(tf.keras.layers.Dense(units=hn_neurons[2], activation=ac_fun[2]))
net.add(tf.keras.layers.Dense(units=ou_neurons   , activation=ac_fun[3]))

net.compile(optimizer = op_val, loss = ls_fun, metrics=['accuracy'])

# $$\sigma(\mathbf{x})_{i}=\frac{e^{x_{i}}}{\sum_{j=1}^{K} e^{x_{j}}}$$


In [ ]:
#@title Network summary
net.summary()

In [ ]:
#@title Train the network
history = net.fit(datain_tr_calibrated,
                  dataou_tr_categorical,
                  epochs     = it_val ,
                  batch_size = bt_size,
                  verbose    = vr_val ,
                  shuffle    = sh_val)

In [ ]:
#@title Estimate outputs of testing data
dataes_te_categorical = net.predict(datain_te_calibrated)
dataes_te_categorical

In [ ]:
#@title Define an accuracy function
def fun_accuracy(dataes, dataou):
  num_err = np.count_nonzero( dataes.argmax(axis = 1) - dataou.argmax(axis = 1) )
  accuracy = 1 - num_err/dataou.shape[0]
  return accuracy


In [ ]:
#@title Compute accuracy
print('testing accuracy: {} %'.format( fun_accuracy(dataes_te_categorical,dataou_te_categorical)*100) )

In [ ]:
#@title Plot iterations vs losses
import matplotlib.pyplot as plt

plt.figure(figsize=[8,8])
plt.plot(history.history['loss'],'--')
plt.title('Model Loss', fontsize = 20)
plt.ylabel('Loss', fontsize = 20)
plt.xlabel('Iteration Number', fontsize = 20)


In [ ]:
#@title Plot iterations vs accuracy
plt.figure(figsize=[8,8])
plt.plot(history.history['accuracy'],'--')
plt.title('Model Accuracy', fontsize = 20)
plt.ylabel('Accuracy', fontsize = 20)
plt.xlabel('Iteration Number', fontsize = 20)


# Lecture 11: TensorFlow Keras for Classification Neural Networks
In this lecture, you learned about:
1. How to develop a shallow neural network using TensorFlow Keras in Python for a classification problem
2. How to develop a deep neural network using TensorFlow Keras in Python for a classification problem

***In the next lecture, we will learn about how we can Increase Our Python Expertise and Knowlege on Our Own!***
